# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

Neuron: a container that holds a single scalar (usually a floating-point number) called an activation; neurons combine to form columns that make up either the input data or layers in a neural network. Also called 'node'.   
Input Layer: Ryan Allred mentioned during lecture that sometimes 'layer' is a misnomer with regard to input. In any case, this is the left-most information in a NN and is in the form of an array of rows by columns.  
Hidden Layer: this is a true layer, comprised of neurons, of which at least 1 is required to form a non-perceptron NN. Each hidden layer in a NN receives activations from the input or previous hidden layer, applies a weight to each activation then a bias, and forwards the new activations to each neuron in the next hidden layer or output layer.  
Output Layer: the right-most layer in a NN, this receives activations from the previous layer then returns final scalars, either integer or float, that provide an information array about the question of interest, eg, classification or regression.  
Activation: a single scalar found in a neuron.  
Backpropagation: an algorithmic process by which weights in a NN are revised in a backwards propagated fashion after each training epoch, ie last weight is revised, then one just prior, then one before that, etc.

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [1]:
import numpy as np

# Define x1, x2, x3

x1 = [1, 1, 0, 1]
x2 = [1, 0, 1, 0]
x3 = [1, 1, 1, 1]
y_list = [1, 0, 0, 0]

X = np.array(list(zip(x1, x2, x3, np.ones(4))))
y = np.array([[val] for val in y_list])
X.shape, y.shape

((4, 4), (4, 1))

In [2]:
# Write perceptron class


class ANDPerceptron():
    def __init__(self, X, y, niter=100):
        self.X = X
        self.y = y
        self.niter = niter
       
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_prime(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def fit(self):
        # Create weights
        weights = 2 * np.random.random((self.X.shape[0], self.X.shape[1])) - 1

        for iteration in range(self.niter):
            # Weighted sum of inputs and weights
            weighted_sum = np.dot(self.X, weights)

            # Activate with sigmoid function
            activated_output = self.sigmoid(weighted_sum)

            # Calculate Error
            error = self.y - activated_output

            # Calculate weight adjustments with sigmoid_derivative
            adjustments = error * self.sigmoid_prime(activated_output)

            # Update weights
            weights += np.dot(self.X.T, adjustments)
        
        print('optimized weights after training: ')
        print(weights)
        print('\ny:', y)
        print("\noutputs after training:")
        print(activated_output)

In [3]:
AP = ANDPerceptron(X, y)

In [4]:
AP.fit()

optimized weights after training: 
[[ 2.92611756  2.89688383  3.16407008  2.95721367]
 [ 3.74074392  3.70016481  3.93126604  3.76444709]
 [-3.15047259 -3.19682681 -2.93397234 -1.87306245]
 [-2.40487168 -2.30702194 -2.93880274 -3.7228538 ]]

y: [[1]
 [0]
 [0]
 [0]]

outputs after training:
[[0.75048007 0.74700151 0.77085915 0.75315822]
 [0.06783945 0.06928197 0.06297163 0.06724257]
 [0.14130851 0.14263029 0.12644057 0.13924864]
 [0.06783945 0.06928197 0.06297163 0.06724257]]


## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [5]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')

In [6]:
df.isna().sum().sum(), df.head()

(0,
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
 0   63    1   3       145   233    1        0      150      0      2.3      0   
 1   37    1   2       130   250    0        1      187      0      3.5      0   
 2   41    0   1       130   204    0        0      172      0      1.4      2   
 3   56    1   1       120   236    0        1      178      0      0.8      2   
 4   57    0   0       120   354    0        1      163      1      0.6      2   
 
    ca  thal  target  
 0   0     1       1  
 1   0     2       1  
 2   0     2       1  
 3   0     2       1  
 4   0     2       1  )

In [19]:
X = df.drop('target', axis=1)
y = df.target
y = np.array(y).reshape(-1, 1)
X.shape, y.shape

((303, 13), (303, 1))

In [15]:
class MultilayerPerceptron():
    def __init__(self):
        '''
        Define node size of input, hidden layer (one hidden layer only here), and output layer;
        plus weights (two). These values are all fixed
        '''
        self.input_size = 13
        self.hidden_layer_size = 4
        self.output_layer_size = 1
        
        # Weights (parameters)
        self.L1_weights = np.random.randn(self.input_size, self.hidden_layer_size)
        self.L2_weights = np.random.randn(self.hidden_layer_size, self.output_layer_size)
    
    def forward(self, X):
        '''Propagate inputs forward through network'''
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)  # WL calls this self.z2; summation is the idea
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)  # WL calls this self.a2
        # Weighted sum between hidden layer and output layer
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)  # WL calls this self.z3
        y_hat = self.sigmoid(self.output_sum)  # called y_hat because is an estimate of output data 
        return y_hat
    
    def sigmoid(self, s):
        '''Apply sigmoid activation function to scalar, vector, or matrix'''
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_prime(self, s):
        '''Calculate gradient of sigmoid'''
        return np.exp(-s) / ((1 + np.exp(-s))**2)
            
    def cost_function(self, X, y):
        '''
        Compute cost for given X, y, using weights already stored in class. Cost is a
        measure of how incorrect model is after at least one complete forward propagation
        '''
        self.y_hat = self.forward(X)
        J = 0.5 * sum((y - self.y_hat)**2)  # J is term for cost output unit
        return J
        
    def cost_function_prime(self, X, y):
        '''Compute derivative with respect to L1_weights and L2_weights for a given X and y'''
        self.y_hat = self.forward(X)
        
        delta3 = np.multiply(-(y - self.y_hat), self.sigmoid_prime(self.output_sum))
        dJdL2 = np.dot(self.activated_hidden.T, delta3)
        
        delta2 = np.dot(delta3, self.L2_weights.T) * self.sigmoid_prime(self.hidden_sum)
        dJdL1 = np.dot(X.T, delta2)  
        
        return dJdL1, dJdL2
    
    # Helper Functions for interacting with other classes
    def get_params(self):
        '''Get L1_weights and L2_weights unrolled into vector'''
        params = np.concatenate((self.L1_weights.ravel(), self.L2_weights.ravel()))
        return params
    
    def set_params(self, params):
        '''Set L1 and L2 using single parameter vector'''
        L1_start = 0
        L1_end = self.hidden_layer_size * self.input_size
        self.L1_weights = np.reshape(params[L1_start: L1_end], (self.input_size, self.hidden_layer_size))
        L2_end = L1_end + self.hidden_layer_size * self.output_layer_size
        self.L2_weights = np.reshape(params[L1_end: L2_end], (self.hidden_layer_size, self.output_layer_size))
    
    def compute_gradients(self, X, y):
        '''
        Returns the vector that takes us in the most downward direction along some function
        in hyperspace that has as many dimensions as we have weights--2, in this case
        '''
        dJdL1, dJdL2 = self.cost_function_prime(X, y)
        return np.concatenate((dJdL1.ravel(), dJdL2.ravel()))

In [9]:
from scipy import optimize

# Make trainer class - credit to Welch Labs

class trainer():
    def __init__(self, N):
        # Make Local reference to network
        self.N = N
    
    def callback_func(self, params):
        self.N.set_params(params)
        self.J.append(self.N.cost_function(self.X, self.y))   
        
    def cost_function_wrapper(self, params, X, y):
        self.N.set_params(params)
        cost = self.N.cost_function(X, y)
        grad = self.N.compute_gradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        # Make an internal variable for the callback function
        self.X = X
        self.y = y

        # Make empty list to store costs
        self.J = []
        
        params0 = self.N.get_params()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.cost_function_wrapper, params0, jac=True, method='BFGS', \
                                 args=(X, y), options=options, callback=self.callback_func)

        self.N.set_params(_res.x)
        self.optimization_results = _res

In [20]:
MLP = MultilayerPerceptron()

In [21]:
T = trainer(MLP)

In [22]:
T.train(X, y)

         Current function value: 38.796519
         Iterations: 0
         Function evaluations: 18
         Gradient evaluations: 6


C:\Users\jhump\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: overflow encountered in square
C:\Users\jhump\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
C:\Users\jhump\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: overflow encountered in square
C:\Users\jhump\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [28]:
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [25]:
# Will continue to work with UCI Heart Disease dataset [domain favorite: health binary classification]
# Train baseline model with 24 input nodes, 1 hidden layer of sixteen nodes, and 1 output node

# Global hyperparameters
inputs = X.shape[1]
epochs = 20
batch_size = 80

# Create model
model = Sequential()
model.add(Dense(24, activation='relu', input_shape=(inputs,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
model.summary()
model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 24)                336       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                400       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 753
Trainable params: 753
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 242 samples, validate on 61 samples
Epoch 1/20
242/242 [==============================] - 1s 4ms/step - loss: 5.0726 - acc: 0.6818 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 2/20
242/242 [==============================] - 0s 50us/step - loss: 5.0726 - acc: 0.6818 - val_loss

In [30]:
# Bring in GridSearchCV, which requires KerasClassifier

# Function to create model, required for KerasClassifier


def create_model():
    # Create model
    model = Sequential()
    model.add(Dense(24, input_dim=13, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


kc_model = KerasClassifier(build_fn=create_model, verbose=1)

# Define the grid search parameters
param_grid = {'batch_size': [20, 40, 60, 80],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=kc_model, param_grid=param_grid, scoring='accuracy', n_jobs=1, cv=5)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# Comment: This model achieved highest performance in my experiments in this notebook.

Epoch 1/20
242/242 [==============================] - 1s 3ms/step - loss: 6.8042 - acc: 0.5702
Epoch 2/20
242/242 [==============================] - 0s 134us/step - loss: 4.4126 - acc: 0.5744
Epoch 3/20
242/242 [==============================] - 0s 145us/step - loss: 3.0221 - acc: 0.6322
Epoch 4/20
242/242 [==============================] - 0s 163us/step - loss: 2.0247 - acc: 0.6364
Epoch 5/20
242/242 [==============================] - 0s 171us/step - loss: 1.3814 - acc: 0.6240
Epoch 6/20
242/242 [==============================] - 0s 153us/step - loss: 0.7759 - acc: 0.6694
Epoch 7/20
242/242 [==============================] - 0s 169us/step - loss: 0.6909 - acc: 0.6983
Epoch 8/20
242/242 [==============================] - 0s 355us/step - loss: 0.6555 - acc: 0.7025
Epoch 9/20
242/242 [==============================] - 0s 192us/step - loss: 0.6909 - acc: 0.6529
Epoch 10/20
242/242 [==============================] - 0s 143us/step - loss: 0.7191 - acc: 0.6364
Epoch 11/20
242/242 [==========

243/243 [==============================] - 0s 43us/step
Epoch 1/20
243/243 [==============================] - 1s 3ms/step - loss: 9.4182 - acc: 0.3169
Epoch 2/20
243/243 [==============================] - 0s 132us/step - loss: 2.4989 - acc: 0.5885
Epoch 3/20
243/243 [==============================] - 0s 121us/step - loss: 1.6997 - acc: 0.6584
Epoch 4/20
243/243 [==============================] - 0s 130us/step - loss: 1.4337 - acc: 0.6296
Epoch 5/20
243/243 [==============================] - 0s 132us/step - loss: 1.3768 - acc: 0.6255
Epoch 6/20
243/243 [==============================] - 0s 134us/step - loss: 1.0702 - acc: 0.6749
Epoch 7/20
243/243 [==============================] - 0s 126us/step - loss: 0.9581 - acc: 0.6708
Epoch 8/20
243/243 [==============================] - 0s 126us/step - loss: 0.8957 - acc: 0.6955
Epoch 9/20
243/243 [==============================] - 0s 136us/step - loss: 0.7775 - acc: 0.6955
Epoch 10/20
243/243 [==============================] - 0s 144us/step - lo

242/242 [==============================] - 0s 50us/step
Epoch 1/20
243/243 [==============================] - 1s 4ms/step - loss: 10.9444 - acc: 0.3210
Epoch 2/20
243/243 [==============================] - 0s 82us/step - loss: 10.9444 - acc: 0.3210
Epoch 3/20
243/243 [==============================] - 0s 76us/step - loss: 10.9444 - acc: 0.3210
Epoch 4/20
243/243 [==============================] - 0s 95us/step - loss: 10.9444 - acc: 0.3210
Epoch 5/20
243/243 [==============================] - 0s 88us/step - loss: 10.9444 - acc: 0.3210
Epoch 6/20
243/243 [==============================] - 0s 84us/step - loss: 10.9444 - acc: 0.3210
Epoch 7/20
243/243 [==============================] - 0s 80us/step - loss: 10.9444 - acc: 0.3210
Epoch 8/20
243/243 [==============================] - 0s 88us/step - loss: 10.9444 - acc: 0.3210
Epoch 9/20
243/243 [==============================] - 0s 78us/step - loss: 10.9444 - acc: 0.3210
Epoch 10/20
243/243 [==============================] - 0s 97us/step - lo

Epoch 20/20
242/242 [==============================] - 0s 35us/step
Epoch 1/20
242/242 [==============================] - 1s 5ms/step - loss: 8.1257 - acc: 0.4959
Epoch 2/20
242/242 [==============================] - 0s 91us/step - loss: 8.1257 - acc: 0.4959
Epoch 3/20
242/242 [==============================] - 0s 68us/step - loss: 8.1257 - acc: 0.4959
Epoch 4/20
242/242 [==============================] - 0s 72us/step - loss: 8.1257 - acc: 0.4959
Epoch 5/20
242/242 [==============================] - 0s 68us/step - loss: 8.1257 - acc: 0.4959
Epoch 6/20
242/242 [==============================] - 0s 74us/step - loss: 8.1257 - acc: 0.4959
Epoch 7/20
242/242 [==============================] - 0s 68us/step - loss: 8.1257 - acc: 0.4959
Epoch 8/20
242/242 [==============================] - 0s 66us/step - loss: 8.1257 - acc: 0.4959
Epoch 9/20
242/242 [==============================] - 0s 70us/step - loss: 8.1257 - acc: 0.4959
Epoch 10/20
242/242 [==============================] - 0s 72us/step -

242/242 [==============================] - 0s 33us/step
Epoch 1/20
242/242 [==============================] - 1s 6ms/step - loss: 6.9268 - acc: 0.5702
Epoch 2/20
242/242 [==============================] - 0s 116us/step - loss: 6.9268 - acc: 0.5702
Epoch 3/20
242/242 [==============================] - 0s 510us/step - loss: 6.9268 - acc: 0.5702
Epoch 4/20
242/242 [==============================] - 0s 85us/step - loss: 6.9268 - acc: 0.5702
Epoch 5/20
242/242 [==============================] - 0s 62us/step - loss: 6.9268 - acc: 0.5702
Epoch 6/20
242/242 [==============================] - 0s 76us/step - loss: 6.9268 - acc: 0.5702
Epoch 7/20
242/242 [==============================] - 0s 79us/step - loss: 6.9268 - acc: 0.5702
Epoch 8/20
242/242 [==============================] - 0s 72us/step - loss: 6.9268 - acc: 0.5702
Epoch 9/20
242/242 [==============================] - 0s 60us/step - loss: 6.9268 - acc: 0.5702
Epoch 10/20
242/242 [==============================] - 0s 60us/step - loss: 6.9

243/243 [==============================] - 0s 95us/step - loss: 0.8426 - acc: 0.6955
Epoch 20/20
243/243 [==============================] - 0s 33us/step


C:\Users\jhump\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/20
303/303 [==============================] - 1s 5ms/step - loss: 6.9331 - acc: 0.3960
Epoch 2/20
303/303 [==============================] - 0s 150us/step - loss: 4.5054 - acc: 0.4059
Epoch 3/20
303/303 [==============================] - 0s 142us/step - loss: 3.2680 - acc: 0.3927
Epoch 4/20
303/303 [==============================] - 0s 135us/step - loss: 2.2994 - acc: 0.3795
Epoch 5/20
303/303 [==============================] - 0s 152us/step - loss: 1.5262 - acc: 0.3696
Epoch 6/20
303/303 [==============================] - 0s 134us/step - loss: 1.1089 - acc: 0.4158
Epoch 7/20
303/303 [==============================] - 0s 144us/step - loss: 0.9067 - acc: 0.5215
Epoch 8/20
303/303 [==============================] - 0s 152us/step - loss: 0.8014 - acc: 0.5248
Epoch 9/20
303/303 [==============================] - 0s 145us/step - loss: 0.7650 - acc: 0.5908
Epoch 10/20
303/303 [==============================] - 0s 137us/step - loss: 0.9241 - acc: 0.5446
Epoch 11/20
303/303 [==========

In [31]:
# Repeat experiment, changing only activation function in output layer from sigmoid to relu

def create_model():
    # Create model
    model = Sequential()
    model.add(Dense(24, input_dim=13, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='relu'))  # changed from sigmoid in previous experiment
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


kc_model = KerasClassifier(build_fn=create_model, verbose=1)

# Create Grid Search
grid = GridSearchCV(estimator=kc_model, param_grid=param_grid, scoring='accuracy', n_jobs=1, cv=5)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# Comment: Changing output layer activation function from sigmoid to relu significantly decreased performance.

Epoch 1/20
242/242 [==============================] - 2s 9ms/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 2/20
242/242 [==============================] - 0s 244us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 3/20
242/242 [==============================] - 0s 227us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 4/20
242/242 [==============================] - 0s 298us/step - loss: 9.0911 - acc: 0.0000e+00 0s - loss: 9.1004 - acc: 0.0000e+00
Epoch 5/20
242/242 [==============================] - 0s 318us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 6/20
242/242 [==============================] - 0s 256us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 7/20
242/242 [==============================] - 0s 200us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 8/20
242/242 [==============================] - 0s 171us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 9/20
242/242 [==============================] - 0s 192us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 10/20
242/242 [==============================] - 0s

243/243 [==============================] - 0s 274us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 20/20
243/243 [==============================] - 0s 152us/step
Epoch 1/20
243/243 [==============================] - 2s 9ms/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 2/20
243/243 [==============================] - 0s 181us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 3/20
243/243 [==============================] - 0s 175us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 4/20
243/243 [==============================] - 0s 160us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 5/20
243/243 [==============================] - 0s 167us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 6/20
243/243 [==============================] - 0s 198us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 7/20
243/243 [==============================] - 0s 265us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 8/20
243/243 [==============================] - 0s 220us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 9/20
243/243 [===============

242/242 [==============================] - 0s 176us/step - loss: 7.9053 - acc: 0.0000e+00
Epoch 17/20
242/242 [==============================] - 0s 138us/step - loss: 7.9053 - acc: 0.0000e+00
Epoch 18/20
242/242 [==============================] - 0s 132us/step - loss: 7.9053 - acc: 0.0000e+00
Epoch 19/20
242/242 [==============================] - 0s 132us/step - loss: 7.9053 - acc: 0.0000e+00
Epoch 20/20
242/242 [==============================] - 0s 56us/step
Epoch 1/20
243/243 [==============================] - 2s 8ms/step - loss: 4.8632 - acc: 0.0329
Epoch 2/20
243/243 [==============================] - 0s 93us/step - loss: 5.0517 - acc: 0.0041
Epoch 3/20
243/243 [==============================] - 0s 99us/step - loss: 5.0527 - acc: 0.0041
Epoch 4/20
243/243 [==============================] - 0s 115us/step - loss: 5.0517 - acc: 0.0041
Epoch 5/20
243/243 [==============================] - 0s 101us/step - loss: 5.0517 - acc: 0.0041
Epoch 6/20
243/243 [==============================] - 0

242/242 [==============================] - 0s 99us/step - loss: 9.0252 - acc: 0.0041
Epoch 16/20
242/242 [==============================] - 0s 118us/step - loss: 9.0252 - acc: 0.0041
Epoch 17/20
242/242 [==============================] - 0s 87us/step - loss: 9.0252 - acc: 0.0041
Epoch 18/20
242/242 [==============================] - 0s 105us/step - loss: 9.0252 - acc: 0.0041
Epoch 19/20
242/242 [==============================] - 0s 87us/step - loss: 9.0252 - acc: 0.0041
Epoch 20/20
242/242 [==============================] - 0s 58us/step
Epoch 1/20
242/242 [==============================] - 2s 9ms/step - loss: 8.1257 - acc: 0.4959
Epoch 2/20
242/242 [==============================] - 0s 78us/step - loss: 8.1257 - acc: 0.4959
Epoch 3/20
242/242 [==============================] - 0s 95us/step - loss: 8.1257 - acc: 0.4959
Epoch 4/20
242/242 [==============================] - 0s 91us/step - loss: 8.1257 - acc: 0.4959
Epoch 5/20
242/242 [==============================] - 0s 91us/step - loss:

242/242 [==============================] - 0s 107us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 14/20
242/242 [==============================] - 0s 120us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 15/20
242/242 [==============================] - 0s 110us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 16/20
242/242 [==============================] - 0s 97us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 17/20
242/242 [==============================] - 0s 89us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 18/20
242/242 [==============================] - 0s 97us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 19/20
242/242 [==============================] - 0s 99us/step - loss: 9.0911 - acc: 0.0000e+00
Epoch 20/20
242/242 [==============================] - 0s 37us/step
Epoch 1/20
242/242 [==============================] - 2s 10ms/step - loss: 6.9268 - acc: 0.5702
Epoch 2/20
242/242 [==============================] - 0s 74us/step - loss: 6.9268 - acc: 0.5702
Epoch 3/20
242/242 [======================

243/243 [==============================] - 0s 93us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 11/20
243/243 [==============================] - 0s 103us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 12/20
243/243 [==============================] - 0s 88us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 13/20
243/243 [==============================] - 0s 76us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 14/20
243/243 [==============================] - 0s 91us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 15/20
243/243 [==============================] - 0s 121us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 16/20
243/243 [==============================] - 0s 95us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 17/20
243/243 [==============================] - 0s 99us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 18/20
243/243 [==============================] - 0s 91us/step - loss: 5.1173 - acc: 0.0000e+00
Epoch 19/20
243/243 [==============================] - 0s 113us/step - loss: 5.1173 - acc: 0.0000e+00

C:\Users\jhump\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/20
303/303 [==============================] - 3s 9ms/step - loss: 7.2101 - acc: 0.0066
Epoch 2/20
303/303 [==============================] - 0s 83us/step - loss: 7.2090 - acc: 0.0066
Epoch 3/20
303/303 [==============================] - 0s 106us/step - loss: 7.2123 - acc: 0.0000e+00
Epoch 4/20
303/303 [==============================] - 0s 84us/step - loss: 7.2083 - acc: 0.0033
Epoch 5/20
303/303 [==============================] - 0s 112us/step - loss: 7.2083 - acc: 0.0033
Epoch 6/20
303/303 [==============================] - 0s 101us/step - loss: 7.2083 - acc: 0.0033
Epoch 7/20
303/303 [==============================] - 0s 94us/step - loss: 7.1195 - acc: 0.0099
Epoch 8/20
303/303 [==============================] - 0s 101us/step - loss: 7.1123 - acc: 0.0165
Epoch 9/20
303/303 [==============================] - 0s 94us/step - loss: 7.2083 - acc: 0.0033
Epoch 10/20
303/303 [==============================] - 0s 82us/step - loss: 7.2083 - acc: 0.0033
Epoch 11/20
303/303 [===========

In [32]:
# Train new model with 36 input nodes, 1 hidden layer of sixteen nodes, and 1 output node

# Global hyperparameters
inputs = X.shape[1]
epochs = 20
batch_size = 80

# Create model
model = Sequential()
model.add(Dense(36, activation='relu', input_shape=(inputs,)))  # was 24 input nodes in baseline
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
model.summary()
model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size)

# Comment: Increasing input nodes from 24 to 36 worsened the model's performance

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_133 (Dense)            (None, 36)                504       
_________________________________________________________________
dense_134 (Dense)            (None, 16)                592       
_________________________________________________________________
dense_135 (Dense)            (None, 1)                 17        
Total params: 1,113
Trainable params: 1,113
Non-trainable params: 0
_________________________________________________________________
Train on 242 samples, validate on 61 samples
Epoch 1/20
242/242 [==============================] - 4s 15ms/step - loss: 3.5429 - acc: 0.3636 - val_loss: 9.3469 - val_acc: 0.0000e+00
Epoch 2/20
242/242 [==============================] - 0s 132us/step - loss: 3.1809 - acc: 0.6818 - val_loss: 9.5680 - val_acc: 0.0000e+00
Epoch 3/20
242/242 [==============================] - 0s 172us/step - loss: 2.3019 -

In [33]:
# Train new model with 24 input nodes [same as baseline], 1 hidden layer of 24 nodes [up from baseline],
# and 1 output node

# Global hyperparameters
inputs = X.shape[1]
epochs = 20
batch_size = 80

# Create model
model = Sequential()
model.add(Dense(24, activation='relu', input_shape=(inputs,)))
model.add(Dense(24, activation='relu'))  # was 16 nodes at baseline
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
model.summary()
model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size)

# Comment: Increasing hidden layer nodes from 16 to 24 decreased the model's performance.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_136 (Dense)            (None, 24)                336       
_________________________________________________________________
dense_137 (Dense)            (None, 24)                600       
_________________________________________________________________
dense_138 (Dense)            (None, 1)                 25        
Total params: 961
Trainable params: 961
Non-trainable params: 0
_________________________________________________________________
Train on 242 samples, validate on 61 samples
Epoch 1/20
242/242 [==============================] - 4s 15ms/step - loss: 4.5166 - acc: 0.6818 - val_loss: 9.4226 - val_acc: 0.0164
Epoch 2/20
242/242 [==============================] - 0s 101us/step - loss: 2.5608 - acc: 0.5331 - val_loss: 2.1147 - val_acc: 0.4918
Epoch 3/20
242/242 [==============================] - 0s 101us/step - loss: 2.8987 - acc: 0.3223